<a href="https://colab.research.google.com/github/erico-bachour/Challenge_Data_Science_2_Alura/blob/main/Insight_Places_3_Clusteriza%C3%A7%C3%A3o_e_Recomendador_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Insight Places 3 - Clusterização e Recomendador**

Nesta etapa do projeto para a Imobiliária Insight Places vamos partir para uma outra forma de modelagem. Com a base de dados tratada na fase anterior, iremos padronizá-la para que possamos dividir os imóveis em grupos. Dessa forma, o objetivo é criar um recomendador que, a partir de um imóvel escolhido, indique-nos uma série de imóveis de características semelhantes.

##1. Preparação do ambiente de trabalho e download da base de dados

In [2]:
!pip install pyspark==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 58.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=8df596fde43d7ad3b1f198289d4135270101f1d5a63a1d3d4b592a98f6946c9a
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Insight Places 3") \
    .getOrCreate()

spark

In [4]:
import zipfile
!wget 'https://caelum-online-public.s3.amazonaws.com/challenge-spark/semanas-3-e-4.zip'

--2022-12-13 16:47:05--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semanas-3-e-4.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 52.217.44.164, 52.217.231.177, 52.217.192.169, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|52.217.44.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2588308 (2.5M) [application/zip]
Saving to: ‘semanas-3-e-4.zip’

semanas-3-e-4.zip   100%[===================>]   2.47M  5.71MB/s    in 0.4s    

2022-12-13 16:47:06 (5.71 MB/s) - ‘semanas-3-e-4.zip’ saved [2588308/2588308]



In [5]:
!unzip -o semanas-3-e-4.zip -d ./

Archive:  semanas-3-e-4.zip
   creating: ./dataset_ml_parquet/
  inflating: ./dataset_ml_parquet/_SUCCESS  
  inflating: ./dataset_ml_parquet/._SUCCESS.crc  
  inflating: ./dataset_ml_parquet/part-00003-a14b227c-f87e-4893-b5f9-4163ed07cb37-c000.snappy.parquet  
  inflating: ./dataset_ml_parquet/part-00001-a14b227c-f87e-4893-b5f9-4163ed07cb37-c000.snappy.parquet  
  inflating: ./dataset_ml_parquet/part-00000-a14b227c-f87e-4893-b5f9-4163ed07cb37-c000.snappy.parquet  
  inflating: ./dataset_ml_parquet/.part-00003-a14b227c-f87e-4893-b5f9-4163ed07cb37-c000.snappy.parquet.crc  
  inflating: ./dataset_ml_parquet/part-00002-a14b227c-f87e-4893-b5f9-4163ed07cb37-c000.snappy.parquet  
  inflating: ./dataset_ml_parquet/.part-00001-a14b227c-f87e-4893-b5f9-4163ed07cb37-c000.snappy.parquet.crc  
  inflating: ./dataset_ml_parquet/.part-00002-a14b227c-f87e-4893-b5f9-4163ed07cb37-c000.snappy.parquet.crc  
  inflating: ./dataset_ml_parquet/.part-00000-a14b227c-f87e-4893-b5f9-4163ed07cb37-c000.snappy.parq

In [6]:
dataset = spark.read.parquet('/content/dataset_ml_parquet')

Mostrando a base de dados e o número de registros no arquivo

In [7]:
dataset.limit(5).show()
print('Número de linhas no arquivo: %s' % dataset.count())

+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|      bairro|condominio| iptu|   valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|00002dd9-cc74-480...|    2|       35|        1|      1|   0.0| 0.0|Santo Cristo|     100.0|100.0|245000.0|           1|         0|     

Mostrando os tipos dos dados:

In [8]:
dataset.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: double (nullable = true)
 |-- vaga: double (nullable = true)
 |-- bairro: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- valor: double (nullable = true)
 |-- Zona Central: integer (nullable = true)
 |-- Zona Norte: integer (nullable = true)
 |-- Zona Oeste: integer (nullable = true)
 |-- Zona Sul: integer (nullable = true)
 |-- Academia: integer (nullable = true)
 |-- Animais permitidos: integer (nullable = true)
 |-- Churrasqueira: integer (nullable = true)
 |-- Condomínio fechado: integer (nullable = true)
 |-- Elevador: integer (nullable = true)
 |-- Piscina: integer (nullable = true)
 |-- Playground: integer (nullable = true)
 |-- Portaria 24h: integer (nullable = true)
 |-- Portão eletrônico: integer (nullable 

Como padrão nos modelos de machine learning, vamos transformar a coluna 'valor' em 'label'.

In [9]:
dataset = dataset\
        .withColumnRenamed('valor', 'label')

In [10]:
dataset.columns

['id',
 'andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'bairro',
 'condominio',
 'iptu',
 'label',
 'Zona Central',
 'Zona Norte',
 'Zona Oeste',
 'Zona Sul',
 'Academia',
 'Animais permitidos',
 'Churrasqueira',
 'Condomínio fechado',
 'Elevador',
 'Piscina',
 'Playground',
 'Portaria 24h',
 'Portão eletrônico',
 'Salão de festas']

## 2. Transformação dos Dados

### 2.1. Vector Assembling

Para tornar possivel a modelagem por clustering, vamos transformar as diversas features da base de dados em um vetor somente, por meio da ferramenta Vector Assembler.

In [11]:
from pyspark.ml.feature import VectorAssembler

In [12]:
x = ['andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'condominio',
 'iptu',
 'Zona Central',
 'Zona Norte',
 'Zona Oeste',
 'Zona Sul',
 'Academia',
 'Animais permitidos',
 'Churrasqueira',
 'Condomínio fechado',
 'Elevador',
 'Piscina',
 'Playground',
 'Portaria 24h',
 'Portão eletrônico',
 'Salão de festas']

dataset_vector = VectorAssembler(inputCols = x, outputCol = 'features').transform(dataset).select('features', 'label', 'id')

dataset_vector.show(5, truncate=False)

+------------------------------------------------------------------------------------------------+--------+------------------------------------+
|features                                                                                        |label   |id                                  |
+------------------------------------------------------------------------------------------------+--------+------------------------------------+
|[2.0,35.0,1.0,1.0,0.0,0.0,100.0,100.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]  |245000.0|00002dd9-cc74-4809-b5a5-850adf0e7526|
|(22,[0,1,2,3,5,6,7,9,14,16,18,19,21],[1.0,84.0,2.0,2.0,1.0,770.0,105.0,1.0,1.0,1.0,1.0,1.0,1.0])|474980.0|0009ca94-2b37-4381-b8b8-773ce0f92444|
|(22,[1,2,3,6,7,11,13,16],[85.0,2.0,2.0,460.0,661.0,1.0,1.0,1.0])                                |290000.0|000e3d28-e3e5-4110-b488-69154931140e|
|(22,[1,2,3,5,6,7,10,17,18],[58.0,1.0,2.0,1.0,550.0,550.0,1.0,1.0,1.0])                          |249000.0|000fb707-6cad-496d-8cb7

### 2.2. Standard Scaler - Padronização

Para podermos inserir os dados de maneira correta no modelo, eles devem estar em uma mesma escala. Assim, usaremos a feature Standard Scaler do spark para realizar a tarefa.

In [13]:
from pyspark.ml.feature import StandardScaler

In [14]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

# fitando o normalizador à base de dados
scalerModel = scaler.fit(dataset_vector)

# criando a coluna de features normalizadas
X_normalizado = scalerModel.transform(dataset_vector)

In [15]:
X_normalizado.show()

+--------------------+---------+--------------------+--------------------+
|            features|    label|                  id|      scaledFeatures|
+--------------------+---------+--------------------+--------------------+
|[2.0,35.0,1.0,1.0...| 245000.0|00002dd9-cc74-480...|[0.13607726247524...|
|(22,[0,1,2,3,5,6,...| 474980.0|0009ca94-2b37-438...|(22,[0,1,2,3,5,6,...|
|(22,[1,2,3,6,7,11...| 290000.0|000e3d28-e3e5-411...|(22,[1,2,3,6,7,11...|
|(22,[1,2,3,5,6,7,...| 249000.0|000fb707-6cad-496...|(22,[1,2,3,5,6,7,...|
|(22,[1,2,3,4,5,6,...| 530000.0|001b6db0-e88d-4eb...|(22,[1,2,3,4,5,6,...|
|[0.0,200.0,6.0,4....|2900000.0|002db67e-52cc-480...|[0.0,2.2447697820...|
|(22,[1,2,3,6,9,16...| 389000.0|002dc4e0-f1a5-4ad...|(22,[1,2,3,6,9,16...|
|(22,[0,1,2,3,4,5,...| 470000.0|0030dd84-f657-41f...|(22,[0,1,2,3,4,5,...|
|(22,[1,2,3,4,5,10...|4400000.0|003213bb-ec21-4ad...|(22,[1,2,3,4,5,10...|
|[0.0,41.0,1.0,1.0...| 269900.0|0032884c-aa5f-455...|[0.0,0.4601778053...|
|[5.0,78.0,1.0,2.0...|105

### 2.3. Redução de Dimensionalidade - PCA

Para conseguirmos criar nosso modelo de recomendação, precisamos reduzir a dimensão dos nossos dados, pois atualmente temos 22 features. Para fazermos isso, podemos utilizar a técnica chamada PCA (Principal Component Analisys)

Aplicando essa técnica, teremos um novo conjunto de dados onde as colunas serão uma combinação linear das colunas originais. Assim, essas novas features explicarão boa parte da variância do total, porém trazendo menos confusão ao modelo.

O objetivo é conseguir um número de features que explique ao mínimo 70% da variância das features do modelo original. Assim, teremos menos informações mas manteremos uma boa explicação sobre os resultados.

In [16]:
x

['andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'condominio',
 'iptu',
 'Zona Central',
 'Zona Norte',
 'Zona Oeste',
 'Zona Sul',
 'Academia',
 'Animais permitidos',
 'Churrasqueira',
 'Condomínio fechado',
 'Elevador',
 'Piscina',
 'Playground',
 'Portaria 24h',
 'Portão eletrônico',
 'Salão de festas']

In [17]:
from pyspark.ml.feature import PCA

In [18]:
pca = PCA(k=7, inputCol='scaledFeatures', outputCol='pca_features')
model_pca = pca.fit(X_normalizado)
dataset_pca = model_pca.transform(X_normalizado)

In [19]:
dataset_pca.limit(2).show()

+--------------------+--------+--------------------+--------------------+--------------------+
|            features|   label|                  id|      scaledFeatures|        pca_features|
+--------------------+--------+--------------------+--------------------+--------------------+
|[2.0,35.0,1.0,1.0...|245000.0|00002dd9-cc74-480...|[0.13607726247524...|[-6.1850665561860...|
|(22,[0,1,2,3,5,6,...|474980.0|0009ca94-2b37-438...|(22,[0,1,2,3,5,6,...|[-3.2149125552925...|
+--------------------+--------+--------------------+--------------------+--------------------+



In [20]:
model_pca.explainedVariance

DenseVector([0.2771, 0.1559, 0.0905, 0.0568, 0.0546, 0.0479, 0.0457])

In [21]:
print(model_pca.explainedVariance.sum())

0.7284955171033062


Após testes, chegou-se a conclusão que o número ótimo de features é 7.

##3. Clusterização

Com a base de dados tratada, vetorizada e normalizada, podemos dar prosseguimento a criação do nosso modelo. O que vamos fazer agora chama-se Clusterização, que é um agrupamento por similaridade. Assim, imóveis de características semelhantes ficarão unidos em clusters.

In [22]:
from pyspark.ml.clustering import KMeans

Para que esse projeto possa ser replicado vamos definir uma SEED única

In [23]:
SEED = 1001

Para efeitos de teste, vamos começar com 5 clusters.

In [24]:
kmeans = KMeans(featuresCol='pca_features', predictionCol='prediction').setK(5).setSeed(SEED)

In [25]:
model = kmeans.fit(dataset_pca)

In [26]:
prediction = model.transform(dataset_pca)

In [27]:
prediction.show()

+--------------------+---------+--------------------+--------------------+--------------------+----------+
|            features|    label|                  id|      scaledFeatures|        pca_features|prediction|
+--------------------+---------+--------------------+--------------------+--------------------+----------+
|[2.0,35.0,1.0,1.0...| 245000.0|00002dd9-cc74-480...|[0.13607726247524...|[-6.1850665561860...|         3|
|(22,[0,1,2,3,5,6,...| 474980.0|0009ca94-2b37-438...|(22,[0,1,2,3,5,6,...|[-3.2149125552925...|         0|
|(22,[1,2,3,6,7,11...| 290000.0|000e3d28-e3e5-411...|(22,[1,2,3,6,7,11...|[-1.0068958653993...|         1|
|(22,[1,2,3,5,6,7,...| 249000.0|000fb707-6cad-496...|(22,[1,2,3,5,6,7,...|[-1.9528334788197...|         1|
|(22,[1,2,3,4,5,6,...| 530000.0|001b6db0-e88d-4eb...|(22,[1,2,3,4,5,6,...|[-0.3550159242190...|         0|
|[0.0,200.0,6.0,4....|2900000.0|002db67e-52cc-480...|[0.0,2.2447697820...|[-6.6836779492566...|         2|
|(22,[1,2,3,6,9,16...| 389000.0|002dc

###3.1 Testando a Clusterização - Silhouette

A fim de testarmos o número ótimo de clusters para nossa base de dados, vamos usar o coeficiente de Silhouette, que mede a dispersão entre os pontos de um cluster pela distância média do ponto e os pontos do mesmo cluster e a disância média do ponto e os pontos do cluster mais próximo.

In [28]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [29]:
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(prediction)
print("O Coeficiente de Silhouette é " + str(silhouette))

O Coeficiente de Silhouette é -0.6798058918573732


Agora, vamos calcular o mesmo coeficiente para uma série de números de clusters. Dessa forma, escolheremos um número que consiga explicar as variações na amostra sem comprometer as dispersões entre os pontos.

In [30]:
range_n_clusters = [2,3,4,5,6,7,8,9,10]
for n_clusters in range_n_clusters:

  kmeans = KMeans(featuresCol='pca_features', predictionCol='prediction', seed=SEED).setK(n_clusters)

  model = kmeans.fit(dataset_pca)

  prediction = model.transform(dataset_pca)

  evaluator = ClusteringEvaluator()

  silhouette = evaluator.evaluate(prediction)
  
  print("O Coeficiente de Silhouette com ", n_clusters, " clusters é " + str(silhouette))

O Coeficiente de Silhouette com  2  clusters é 0.030640652769336182
O Coeficiente de Silhouette com  3  clusters é -0.27748011314680937
O Coeficiente de Silhouette com  4  clusters é -0.2019778876886637
O Coeficiente de Silhouette com  5  clusters é -0.6798058918573732
O Coeficiente de Silhouette com  6  clusters é -0.6877653805655075
O Coeficiente de Silhouette com  7  clusters é -0.7079128508028183
O Coeficiente de Silhouette com  8  clusters é -0.473942131903593
O Coeficiente de Silhouette com  9  clusters é -0.6700421821230954
O Coeficiente de Silhouette com  10  clusters é -0.9736366110073471


Vamos ficar com o número de 8 clusters, pois com ele conseguimos dividir a amostra sem comprometer a dispersão.

In [31]:
kmeans = KMeans(featuresCol='pca_features', predictionCol='prediction', seed=SEED).setK(8)

model = kmeans.fit(dataset_pca)

prediction = model.transform(dataset_pca)

prediction.show(5)
print('Número de linhas no arquivo: %s' % prediction.count())

+--------------------+--------+--------------------+--------------------+--------------------+----------+
|            features|   label|                  id|      scaledFeatures|        pca_features|prediction|
+--------------------+--------+--------------------+--------------------+--------------------+----------+
|[2.0,35.0,1.0,1.0...|245000.0|00002dd9-cc74-480...|[0.13607726247524...|[-6.1850665561860...|         3|
|(22,[0,1,2,3,5,6,...|474980.0|0009ca94-2b37-438...|(22,[0,1,2,3,5,6,...|[-3.2149125552925...|         2|
|(22,[1,2,3,6,7,11...|290000.0|000e3d28-e3e5-411...|(22,[1,2,3,6,7,11...|[-1.0068958653993...|         1|
|(22,[1,2,3,5,6,7,...|249000.0|000fb707-6cad-496...|(22,[1,2,3,5,6,7,...|[-1.9528334788197...|         0|
|(22,[1,2,3,4,5,6,...|530000.0|001b6db0-e88d-4eb...|(22,[1,2,3,4,5,6,...|[-0.3550159242190...|         2|
+--------------------+--------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

Número de linhas no a

##4. Filtrar Imóveis

Como solicitado pela esquipe de Data Science da Insight Places, vamos desenvolver um filtro para que, a partir do ID de um imóvel, possamos descobrir imóveis pertencem ao mesmo cluster.

Para fins de exemplo, vamos pegar um dos IDs da base de dados e usá-lo para demonstrar o filtro.

In [32]:
id_imovel = '000e3d28-e3e5-4110-b488-69154931140e'

In [33]:
cluster = prediction.filter(prediction.id == id_imovel).select('prediction').collect()[0][0]
cluster

1

In [34]:
imoveis_recomendados = prediction.filter(prediction.prediction == cluster)\
                                        .select('id', 'pca_features')
imoveis_recomendados.show()

+--------------------+--------------------+
|                  id|        pca_features|
+--------------------+--------------------+
|000e3d28-e3e5-411...|[-1.0068958653993...|
|00370d62-f774-405...|[-0.9112993124695...|
|005c608c-5afc-459...|[-0.2029130286116...|
|008f5865-d5f4-408...|[-1.1813761520667...|
|0092be3c-1736-42a...|[-0.4863135813652...|
|00b61c4e-bfc0-470...|[-2.0953409216706...|
|00cd6c5d-592d-429...|[0.19283085292747...|
|00d01937-9bca-45a...|[-0.4427156106663...|
|00fcb5fb-c01b-44b...|[-0.3410577473496...|
|00ff8cc4-306e-48a...|[0.04154793417111...|
|010cc24b-6cc5-4b8...|[-0.9088326011083...|
|01228c26-3a13-480...|[-1.4641566466594...|
|01259581-b96b-4ce...|[-0.0889017559452...|
|01429b39-66ab-413...|[-0.3387502716969...|
|0148c5de-0b01-45b...|[-0.0167463781491...|
|01606b94-61f5-44d...|[-0.3260683963709...|
|01643d2f-1fbb-429...|[-1.0488992512326...|
|01853018-3666-457...|[-1.0586243714978...|
|0186f486-f98e-4a8...|[-0.8850607564954...|
|01955663-0e7b-441...|[-1.437011

###4.1 Distância Euclidiana

A mecânica para selecionar quais imóveis do cluster serão recomendadas será utilizando a distância dos imóveis em relação às suas componentes.

Para isso, vamos calcular a distância euclidiana para encontrar as 10 melhores recomendações.

In [35]:
componentes_imovel = imoveis_recomendados.filter(imoveis_recomendados.id == id_imovel)\
                                        .select('pca_features').collect()[0][0]
componentes_imovel

DenseVector([-1.0069, 1.4962, 2.6741, 0.0297, -0.0432, 0.9916, -0.4643])

In [39]:
from scipy.spatial.distance import euclidean
from pyspark.sql.types import FloatType
import pyspark.sql.functions as f

In [45]:
def calcula_distance(value):
  return euclidean(componentes_imovel, value)

udf_calcula_distance = f.udf(calcula_distance, FloatType()) 

imoveis_recomendados_dist = imoveis_recomendados.withColumn('Dist', udf_calcula_distance('pca_features'))

recomendados = spark.createDataFrame(imoveis_recomendados_dist.sort('Dist').take(10)).select(['id', 'Dist'])

recomendados.show(truncate=False)

+------------------------------------+-------------------+
|id                                  |Dist               |
+------------------------------------+-------------------+
|000e3d28-e3e5-4110-b488-69154931140e|0.0                |
|2cebd0b6-c6a0-47b9-afda-f6631a83bce7|0.08258937299251556|
|165ec168-5f7a-4398-ad09-79bfd516f8fe|0.13131853938102722|
|52c76ce2-689b-4a9e-8650-1ac6d7d42e29|0.13568685948848724|
|51129053-266a-4aac-8c79-12ebef486af2|0.14148934185504913|
|6287eda0-756f-46a6-85ff-b8cafc12f1b1|0.1428685039281845 |
|bf4c0e86-a5f2-4d5c-954e-97e23852008f|0.1987978219985962 |
|08368119-f5f4-4cf1-bd53-cdcdee123555|0.20707117021083832|
|1456a709-50fb-45de-abeb-76c06bae39aa|0.2156127691268921 |
|6f5cab16-c874-41c5-8f8d-ffb6389488e5|0.21830229461193085|
+------------------------------------+-------------------+



##5. Função Recomendadora

Para que a equipe de Front-End da Insight Places possa desenvolver uma aplicação para o usuário final, vamos combinar as etapas feitas durante essa parte do projeto em uma única função.

Assim, o usuário escolherá o imóvel e a quantidade de recomendações quer receber.

In [44]:
def recomendador(id, numero):

  cluster = prediction.filter(prediction.id == id_imovel).select('prediction').collect()[0][0]
  imoveis_recomendados = prediction.filter(prediction.prediction == cluster)\
                                       .select('id', 'pca_features')
  componenetes_imovel = imoveis_recomendados.filter(imoveis_recomendados.id == id_imovel)\
                                          .select('pca_features').collect()[0][0]

  def calcula_distance(value):
    return euclidean(componenetes_imovel, value)

  udf_calcula_distance = f.udf(calcula_distance, FloatType())

  imoveis_recomendados_dist = imoveis_recomendados.withColumn('Dist', udf_calcula_distance('pca_features'))

  recomendados = spark.createDataFrame(imoveis_recomendados_dist.sort('Dist').take(numero)).select(['id', 'Dist'])

  recomendados.select('id').show(truncate=False)

In [47]:
recomendador('6f5cab16-c874-41c5-8f8d-ffb6389488e5', 7)

+------------------------------------+
|id                                  |
+------------------------------------+
|000e3d28-e3e5-4110-b488-69154931140e|
|2cebd0b6-c6a0-47b9-afda-f6631a83bce7|
|165ec168-5f7a-4398-ad09-79bfd516f8fe|
|52c76ce2-689b-4a9e-8650-1ac6d7d42e29|
|51129053-266a-4aac-8c79-12ebef486af2|
|6287eda0-756f-46a6-85ff-b8cafc12f1b1|
|bf4c0e86-a5f2-4d5c-954e-97e23852008f|
+------------------------------------+



##6. Pipeline

Como forma de construção de conhecimento, a equipe de Data Science da Insight Places solicitou a elaboração de um pipeline de dados. Assim, a partir de uma base de dados, poderão ser feitas transformações aplicadas sequencialmente de forma automatizada.

In [48]:
from pyspark.ml import Pipeline

In [49]:
pca_pipeline = Pipeline(stages=[VectorAssembler(inputCols= x, outputCol='features'),
                                StandardScaler(inputCol='features', outputCol='scaledFeatures'),
                                PCA(k=7, inputCol='scaledFeatures', outputCol='pca_features')])

In [50]:
pca_pipeline_model = pca_pipeline.fit(dataset)

In [51]:
dataset_pca = pca_pipeline_model.transform(dataset)

In [52]:
dataset_pca.limit(5).show()

+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+--------------------+--------------------+--------------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|      bairro|condominio| iptu|   label|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|            features|      scaledFeatures|        pca_features|
+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+-----------

In [54]:
dataset_pca.select('pca_features').limit(5).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------+
|pca_features                                                                                                                                 |
+---------------------------------------------------------------------------------------------------------------------------------------------+
|[-6.185066556186067,-1.5035861420376833,1.4021971813766747,-0.4627462052987944,0.05817557507559954,-2.06050203373801,-5.349447080300572]     |
|[-3.2149125552925533,1.16996715944037,0.5164078437661357,3.2086930069839372,0.08994771500778509,1.1457808725760208,-0.8593347144471947]      |
|[-1.0068958653993225,1.4962100951692308,2.6741317795672224,0.02967168175495706,-0.043207662071430536,0.9916054175401698,-0.4642687245708697] |
|[-1.9528334788197674,2.054292371016602,-1.0129063602744157,-0.17794843497698848,-0.012607180989539779,0.2228967536340269,-0.03361910379

##7. Avaliação dos Clusters

Vamos pegar a base de dados disponibilizada no começo desta etapa do projeto, concatená-la com a coluna de clusters recém criada e analisá-la a partir dos clusters. Dessa froma podemos saber a média de tamanho ou de quartos dos imóveis de cada cluster, por exemplo.

In [87]:
dataset.show(5)

prediction.show(5)

+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|      bairro|condominio| iptu|   label|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|00002dd9-cc74-480...|    2|       35|        1|      1|   0.0| 0.0|Santo Cristo|     100.0|100.0|245000.0|           1|         0|     

In [88]:
dataset_cluster = dataset.join(prediction, on=['id', 'label'], how='outer')
dataset_cluster.show(5)

+--------------------+---------+-----+---------+---------+-------+------+----+------------+----------+------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+--------------------+--------------------+--------------------+----------+
|                  id|    label|andar|area_util|banheiros|quartos|suites|vaga|      bairro|condominio|  iptu|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|            features|      scaledFeatures|        pca_features|prediction|
+--------------------+---------+-----+---------+---------+-------+------+----+------------+----------+------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------

Fazendo uma limpeza em colunas desnecessárias:

In [89]:
dataset_cluster = dataset_cluster.drop('features', 'scaledFeatures','pca_features')\
                                 .withColumnRenamed('prediction', 'cluster')\
                                 .withColumnRenamed('label', 'valor')

dataset_cluster.show(5)

+--------------------+---------+-----+---------+---------+-------+------+----+------------+----------+------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+-------+
|                  id|    valor|andar|area_util|banheiros|quartos|suites|vaga|      bairro|condominio|  iptu|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|cluster|
+--------------------+---------+-----+---------+---------+-------+------+----+------------+----------+------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+-------+
|00002dd9-cc74-480...| 245000.0|    2|       35|        1|      1|   0.0| 0.0|Santo Cristo|     100.0| 100

###7.1 Area Útil

In [102]:
dataset_cluster\
      .groupBy('cluster')\
      .agg(f.round(f.avg('area_util'),2).alias('Área Útil Média (m2)'))\
      .orderBy('cluster')\
      .show()

+-------+--------------------+
|cluster|Área Útil Média (m2)|
+-------+--------------------+
|      0|               90.65|
|      1|               98.66|
|      2|               84.81|
|      3|               82.37|
|      4|               96.67|
|      5|              264.19|
|      6|              229.82|
|      7|               260.3|
+-------+--------------------+



###7.2 Quartos

In [100]:
dataset_cluster\
      .groupBy('cluster')\
      .agg(f.round(f.avg('quartos'),2).alias('Média Quartos'))\
      .orderBy('cluster')\
      .show()

+-------+-------------+
|cluster|Média Quartos|
+-------+-------------+
|      0|         2.48|
|      1|         2.41|
|      2|         2.36|
|      3|         2.35|
|      4|         2.24|
|      5|         3.78|
|      6|         3.77|
|      7|         3.44|
+-------+-------------+



###7.3 Suítes

In [99]:
dataset_cluster\
      .groupBy('cluster')\
      .agg(f.round(f.avg('suites'),2).alias('Média Suítes'))\
      .orderBy('cluster')\
      .show()

+-------+------------+
|cluster|Média Suítes|
+-------+------------+
|      0|         1.0|
|      1|        0.67|
|      2|        0.47|
|      3|        0.92|
|      4|        0.63|
|      5|        2.54|
|      6|        2.93|
|      7|        1.96|
+-------+------------+



###7.4 Valor

In [104]:
dataset_cluster\
      .groupBy('cluster')\
      .agg(f.round(f.avg('valor'),2).alias('Valor Médio (R$)'))\
      .orderBy('cluster')\
      .show()

+-------+----------------+
|cluster|Valor Médio (R$)|
+-------+----------------+
|      0|       864873.79|
|      1|      1203171.69|
|      2|       483681.76|
|      3|       762778.35|
|      4|       1221670.3|
|      5|      3272943.42|
|      6|      2985522.16|
|      7|      3954287.41|
+-------+----------------+



Fazendo uma breve análise com as características destacadas, percebe-se que os 3 ultimos clusters (5, 6 e 7) possuem uma certa discrepância com o restante dos outros clusters. Isso se vê claramente nos números médios de área útil, suítes e no preço médio dos imóveis desses 3 clusters.